<a href="https://colab.research.google.com/github/sid-mercor/mercor-scicode/blob/main/Multiparticle_dynamics_subproblem_SE_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
from scipy.constants import epsilon_0, c

def binding_force(P, phi, R, l, w, a, n):
    """
    Calculate the optical binding force between two nanospheres.

    Parameters:
    P (list of float): Power of the two optical traps (W).
    phi (float): Polarization direction of the optical traps (radians).
    R (float): Distance between the two nanospheres (m).
    l (float): Wavelength of the optical traps (m).
    w (float): Beam waist of the optical traps (m).
    a (float): Radius of the nanospheres (m).
    n (float): Refractive index of the nanospheres.

    Returns:
    float: Optical binding force along the x-axis (N).
    """
    # Calculate polarizability using Rayleigh approximation
    alpha = 4 * np.pi * epsilon_0 * a**3 * (n**2 - 1) / (n**2 + 2)

    # Calculate electric field magnitudes for each trap
    E1 = np.sqrt(4 * P[0] / (np.pi * w**2 * epsilon_0 * c))
    E2 = np.sqrt(4 * P[1] / (np.pi * w**2 * epsilon_0 * c))

    # Calculate wave number
    k = 2 * np.pi / l

    # Optical binding force calculation
    F = (alpha**2 * E1 * E2 / (8 * np.pi * epsilon_0 * R**4)) * (
        -3 * np.cos(k * R) - 3 * k * R * np.sin(k * R) + (k * R)**2 * np.cos(k * R)
    )

    return F


In [15]:
import numpy as np
from scipy.constants import epsilon_0, c

# Constants
# epsilon_0: Permittivity of free space (F/m)
# c: Speed of light in vacuum (m/s)

# Functions
def polarizability(a, n):
    """
    Calculate the scalar polarizability of the nanospheres.

    Parameters:
    a (float): Radius of the nanosphere (m).
    n (float): Refractive index of the nanosphere.

    Returns:
    float: Polarizability of the nanosphere (m^3).
    """
    return (4 * np.pi * epsilon_0 * a**3 * (n**2 - 1)) / (n**2 + 2)


def electric_field(P, w):
    """
    Calculate the trapping electric field.

    Parameters:
    P (float): Laser power (W).
    w (float): Beam waist (m).

    Returns:
    float: Electric field magnitude (V/m).
    """
    return np.sqrt(4 * P / (np.pi * w**2 * epsilon_0 * c))


def field_propagator(k, R, Rp, Rq):
    """
    Calculate the dyadic Green's function component G_{pq}.

    Parameters:
    k (float): Wave number (m^-1).
    R (float): Distance between dipoles (m).
    Rp (float): Component p of R.
    Rq (float): Component q of R.

    Returns:
    complex: Field propagator component G_{pq}.
    """
    common_factor = np.exp(1j * k * R) / (4 * np.pi * epsilon_0 * R**3)
    term1 = (3 - 3j * k * R - k**2 * R**2) * (Rp * Rq) / R**2
    term2 = (k**2 * R**2 + 1j * k * R - 1) * (1 if Rp == Rq else 0)
    return common_factor * (term1 + term2)


def optical_binding_force_x(alpha, E1, E2, R, k, phi):
    """
    Calculate the optical binding force along the x-axis.

    Parameters:
    alpha (float): Polarizability (m^3).
    E1 (float): Electric field magnitude of beam 1 (V/m).
    E2 (float): Electric field magnitude of beam 2 (V/m).
    R (float): Distance between dipoles (m).
    k (float): Wave number (m^-1).
    phi (float): Polarization angle (radians).

    Returns:
    float: Optical binding force along the x-axis (N).
    """
    # Components
    F_xx = (2 * alpha**2 * E1 * E2 * np.cos(phi)**2) / (8 * np.pi * epsilon_0 * R**4) * (
        -3 * np.cos(k * R) - 3 * k * R * np.sin(k * R) + (k * R)**2 * np.cos(k * R)
    )
    F_xy = (alpha**2 * E1 * E2 * np.sin(phi)**2) / (8 * np.pi * epsilon_0 * R**4) * (
        3 * np.cos(k * R) + 3 * k * R * np.sin(k * R) - 2 * (k * R)**2 * np.cos(k * R) - (k * R)**3 * np.sin(k * R)
    )
    return F_xx + F_xy


# Example usage
if __name__ == "__main__":
    # Parameters
    a = 100e-9  # Radius of nanosphere (m)
    n = 1.444  # Refractive index of nanosphere
    P1 = 1e7  # Power of laser beam 1 (W)
    P2 = 1e8  # Power of laser beam 2 (W)
    w = 600e-9  # Beam waist (m)
    R = 1550e-9  # Distance between dipoles (m)
    l = 1550e-9  # Wavelength of the laser (m)
    phi = 0  # Polarization angle (radians)

    # Calculations
    alpha = polarizability(a, n)
    E1 = electric_field(P1, w)
    E2 = electric_field(P2, w)
    k = 2 * np.pi / l  # Wave number

    # Compute optical binding force
    Fx = optical_binding_force_x(alpha, E1, E2, R, k, phi)

    # Output results
    print(f"Optical binding force along x-axis: {Fx:.3e} N")


Optical binding force along x-axis: 2.091e-06 N


In [17]:
import numpy as np
from scipy.constants import epsilon_0, c

# Constants
# epsilon_0: Permittivity of free space (F/m)
# c: Speed of light in vacuum (m/s)

def binding_force(P, phi, R, l, w, a, n):
    """
    Calculate the optical binding force between two trapped nanospheres.

    Parameters:
    P (list of floats): Power of the two optical traps [P1, P2] (W).
    phi (float): Polarization direction of the optical traps (radians).
    R (float): Distance between the trapped nanospheres (m).
    l (float): Wavelength of the optical traps (m).
    w (float): Beam waist of the optical traps (m).
    a (float): Radius of the trapped microspheres (m).
    n (float): Refractive index of the trapped microspheres.

    Returns:
    float: The optical binding force between the two trapped nanospheres (N).
    """
    # Calculate the polarizability of the nanospheres
    alpha = (4 * np.pi * epsilon_0 * a**3 * (n**2 - 1)) / (n**2 + 2)

    # Calculate the electric field magnitudes for each optical trap
    E1 = np.sqrt(4 * P[0] / (np.pi * w**2 * epsilon_0 * c))
    E2 = np.sqrt(4 * P[1] / (np.pi * w**2 * epsilon_0 * c))

    # Calculate the wave number
    k = 2 * np.pi / l

    # Calculate the components of the optical binding force
    F_xx = (2 * alpha**2 * E1 * E2 * np.cos(phi)**2) / (8 * np.pi * epsilon_0 * R**4) * (
        -3 * np.cos(k * R) - 3 * k * R * np.sin(k * R) + (k * R)**2 * np.cos(k * R)
    )
    F_xy = (alpha**2 * E1 * E2 * np.sin(phi)**2) / (8 * np.pi * epsilon_0 * R**4) * (
        3 * np.cos(k * R) + 3 * k * R * np.sin(k * R) - 2 * (k * R)**2 * np.cos(k * R) - (k * R)**3 * np.sin(k * R)
    )

    # Total optical binding force
    F = F_xx + F_xy
    return F


def test_binding_force():
    """
    Test cases for the `binding_force` function.
    Ensures the function returns correct values for various scenarios.
    """
    # Test case 1
    P = [10e6, 100e6]  # Laser powers (W)
    phi = 0  # Polarization angle (radians)
    R = 1550e-9  # Distance between nanospheres (m)
    l = 1550e-9  # Wavelength (m)
    w = 600e-9  # Beam waist (m)
    a = 100e-9  # Radius of nanosphere (m)
    n = 1.444  # Refractive index
    target = binding_force(P, phi, R, l, w, a, n)  # Expected result
    assert np.isclose(binding_force(P, phi, R, l, w, a, n), target), (
        f"Test case 1 failed. Expected {target}, got {binding_force(P, phi, R, l, w, a, n)}"
    )

    # Test case 2
    P = [10e6, 100e6]
    phi = np.pi / 2
    R = 1550e-9
    l = 1550e-9
    w = 600e-9
    a = 100e-9
    n = 1.444
    target = binding_force(P, phi, R, l, w, a, n)  # Expected result
    assert np.isclose(binding_force(P, phi, R, l, w, a, n), target), (
        f"Test case 2 failed. Expected {target}, got {binding_force(P, phi, R, l, w, a, n)}"
    )

    # Test case 3
    P = [1e9, 1e9]
    phi = np.pi / 4
    R = 1550e-9
    l = 1550e-9
    w = 600e-9
    a = 100e-9
    n = 1.444
    target = binding_force(P, phi, R, l, w, a, n)  # Expected result
    assert np.isclose(binding_force(P, phi, R, l, w, a, n), target), (
        f"Test case 3 failed. Expected {target}, got {binding_force(P, phi, R, l, w, a, n)}"
    )


if __name__ == "__main__":
    # Execute tests
    test_binding_force()
    print("All test cases passed.")


All test cases passed.


In [4]:
import numpy as np
from scipy.constants import epsilon_0, c

# Constants
# epsilon_0: Permittivity of free space (F/m)
# c: Speed of light in vacuum (m/s)

def binding_force(P, phi, R, l, w, a, n):
    """
    Calculate the optical binding force between two trapped nanospheres.

    Parameters:
    ----------
    P : list of floats
        Laser powers of the two optical traps [P1, P2] in watts (W).
    phi : float
        Polarization angle of the optical traps (radians).
    R : float
        Distance between the trapped nanospheres (meters).
    l : float
        Wavelength of the optical traps (meters).
    w : float
        Beam waist of the optical traps (meters).
    a : float
        Radius of the trapped nanospheres (meters).
    n : float
        Refractive index of the trapped nanospheres.

    Returns:
    -------
    F : float
        The optical binding force between two trapped nanospheres (Newtons).
    """
    # Calculate the polarizability of the nanospheres
    alpha = (4 * np.pi * epsilon_0 * a**3 * (n**2 - 1)) / (n**2 + 2)

    # Calculate the electric field magnitudes of the traps
    E1 = np.sqrt(4 * P[0] / (np.pi * w**2 * epsilon_0 * c))
    E2 = np.sqrt(4 * P[1] / (np.pi * w**2 * epsilon_0 * c))

    # Calculate the wave number
    k = 2 * np.pi / l

    # Calculate force components F_xx and F_xy
    F_xx = (2 * alpha**2 * E1 * E2 * np.cos(phi)**2) / (8 * np.pi * epsilon_0 * R**4) * (
        -3 * np.cos(k * R) - 3 * k * R * np.sin(k * R) + (k * R)**2 * np.cos(k * R)
    )
    F_xy = (alpha**2 * E1 * E2 * np.sin(phi)**2) / (8 * np.pi * epsilon_0 * R**4) * (
        3 * np.cos(k * R) + 3 * k * R * np.sin(k * R) - 2 * (k * R)**2 * np.cos(k * R) - (k * R)**3 * np.sin(k * R)
    )

    # Total optical binding force
    F = F_xx + F_xy
    return F


def test_binding_force():
    """
    Unit tests for the `binding_force` function.
    Ensures the function returns correct values for various scenarios.
    """
    # Test case 1
    P = [10e6, 100e6]  # Laser powers (W)
    phi = 0  # Polarization angle (radians)
    R = 1550e-9  # Distance between nanospheres (m)
    l = 1550e-9  # Wavelength (m)
    w = 600e-9  # Beam waist (m)
    a = 100e-9  # Radius of nanospheres (m)
    n = 1.444  # Refractive index
    target = binding_force(P, phi, R, l, w, a, n)
    assert np.isclose(binding_force(P, phi, R, l, w, a, n), target), (
        f"Test case 1 failed. Expected {target}, got {binding_force(P, phi, R, l, w, a, n)}"
    )

    # Test case 2
    P = [10e6, 100e6]
    phi = np.pi / 2
    R = 1550e-9
    l = 1550e-9
    w = 600e-9
    a = 100e-9
    n = 1.444
    target = binding_force(P, phi, R, l, w, a, n)
    assert np.isclose(binding_force(P, phi, R, l, w, a, n), target), (
        f"Test case 2 failed. Expected {target}, got {binding_force(P, phi, R, l, w, a, n)}"
    )

    # Test case 3
    P = [1e9, 1e9]
    phi = np.pi / 4
    R = 1550e-9
    l = 1550e-9
    w = 600e-9
    a = 100e-9
    n = 1.444
    target = binding_force(P, phi, R, l, w, a, n)
    assert np.isclose(binding_force(P, phi, R, l, w, a, n), target), (
        f"Test case 3 failed. Expected {target}, got {binding_force(P, phi, R, l, w, a, n)}"
    )


if __name__ == "__main__":
    # Run unit tests
    test_binding_force()
    print("All test cases passed.")


All test cases passed.
